In [14]:
import cv2
import numpy as np

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eyes_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
mouth_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_smile.xml')
teeth_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_smile.xml')

def eye_aspect_ratio(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    
    # Check for division by zero
    if C == 0:
        return 0
    
    ear = (A + B) / (2.0 * C)
    return ear

def detect_blink(eye, threshold=0.2, consecutive_frames=3):
    ear = eye_aspect_ratio(eye)
    return ear < threshold, ear

blink_frames = 0
blink_count = 0

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        
        # Detect eyes
        eyes = eyes_cascade.detectMultiScale(roi_gray)
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (255, 0, 0), 2)
            cv2.putText(roi_color, 'Eye', (ex, ey - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 0), 2)
            
            # Detect blink
            eye_gray = roi_gray[ey:ey+eh, ex:ex+ew]
            is_blink, ear = detect_blink(eye_gray)
            if is_blink:
                blink_frames += 1
                if blink_frames >= 3:
                    blink_count += 1
                    blink_frames = 0
                cv2.putText(roi_color, 'Blink', (ex, ey + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            else:
                blink_frames = 0
        
        # Detect mouths
        mouths = mouth_cascade.detectMultiScale(roi_gray, scaleFactor=1.7, minNeighbors=11)
        for (mx, my, mw, mh) in mouths:
            cv2.rectangle(roi_color, (mx, my), (mx+mw, my+mh), (0, 0, 255), 2)
            cv2.putText(roi_color, 'Mouth', (mx, my - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        # Detect teeth
        teeth = teeth_cascade.detectMultiScale(roi_gray, scaleFactor=1.7, minNeighbors=11)
        for (tx, ty, tw, th) in teeth:
            cv2.rectangle(roi_color, (tx, ty), (tx+tw, ty+th), (0, 255, 255), 2)
            cv2.putText(roi_color, 'Teeth', (tx, ty - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)
    
    # Display frame
    cv2.imshow('Facial Features Detection', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [4]:
import cv2
import numpy as np
import dlib
from scipy.spatial import distance as dist
from imutils import face_utils

def mouth_aspect_ratio(mouth):
    A = dist.euclidean(mouth[2], mouth[10])  # 51, 59
    B = dist.euclidean(mouth[4], mouth[8])  # 53, 57
    C = dist.euclidean(mouth[0], mouth[6])  # 49, 55
    mar = (A + B) / (2.0 * C)
    return mar

def eye_aspect_ratio(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    ear = (A + B) / (2.0 * C)
    return ear



detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

MOUTH_AR_THRESH = 0.5
EYE_AR_THRESH = 0.25
EYE_AR_CONSEC_FRAMES = 3

(mStart, mEnd) = (49, 68)
(lStart, lEnd) = (42, 48)
(rStart, rEnd) = (36, 42)

COUNTER_BLINK = 0
TOTAL_BLINKS = 0

vs = cv2.VideoCapture(0)

while True:
    ret, frame = vs.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    rects = detector(gray, 0)

    for rect in rects:
        (x, y, w, h) = face_utils.rect_to_bb(rect)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        mouth = shape[mStart:mEnd]
        mouthMAR = mouth_aspect_ratio(mouth)

        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        ear = (leftEAR + rightEAR) / 2.0

        mouthHull = cv2.convexHull(mouth)
        cv2.drawContours(frame, [mouthHull], -1, (0, 255, 0), 1)

        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

        if mouthMAR > MOUTH_AR_THRESH:
            cv2.putText(frame, "Mouth is Open!", (30, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        if ear < EYE_AR_THRESH:
            COUNTER_BLINK += 1
            if COUNTER_BLINK >= EYE_AR_CONSEC_FRAMES:
                cv2.putText(frame, "Blink Detected!", (30, 90),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        else:
            if COUNTER_BLINK >= EYE_AR_CONSEC_FRAMES:
                TOTAL_BLINKS += 1
            COUNTER_BLINK = 0

    cv2.imshow("Frame", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

vs.release()
cv2.destroyAllWindows()

In [2]:
pip install imutils

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25854 sha256=620ef1f7cb73e611375c62ab5c82c6e01964cccca00345ce8ad36c1f0944b8c4
  Stored in directory: c:\users\abhia\appdata\local\pip\cache\wheels\31\d0\2c\87ce38f6052879e5b7b18f0f8b4a10ad2a9d210e908d449f16
Successfully built imutils
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import numpy as np
import dlib
from scipy.spatial import distance as dist
from imutils import face_utils

def mouth_aspect_ratio(mouth):
    A = dist.euclidean(mouth[14], mouth[18])  # 65, 67
    B = dist.euclidean(mouth[12], mouth[16])  # 63, 61
    C = dist.euclidean(mouth[0], mouth[6])  # 49, 55
    mar = (A + B) / (2.0 * C)
    return mar

def eye_aspect_ratio(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def detect_blink(eye, threshold=0.2, consecutive_frames=3):
    ear = eye_aspect_ratio(eye)
    return ear < threshold, ear

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eyes_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
mouth_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_smile.xml')
teeth_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_smile.xml')

MOUTH_AR_THRESH = 0.6
EYE_AR_THRESH = 0.25
EYE_AR_CONSEC_FRAMES = 3
TEETH_SCALE_FACTOR = 1.7
TEETH_MIN_NEIGHBORS = 11

(mStart, mEnd) = (49, 68)
(lStart, lEnd) = (42, 48)
(rStart, rEnd) = (36, 42)

COUNTER_BLINK = 0
TOTAL_BLINKS = 0

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Mouth and eyes detection
        rects = detector(gray, 0)
        for rect in rects:
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)

            mouth = shape[mStart:mEnd]
            mouthMAR = mouth_aspect_ratio(mouth)

            leftEye = shape[lStart:lEnd]
            rightEye = shape[rStart:rEnd]
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            ear = (leftEAR + rightEAR) / 2.0

            if mouthMAR > MOUTH_AR_THRESH:
                cv2.putText(frame, "Mouth is Open!", (30, 60),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            if ear < EYE_AR_THRESH:
                COUNTER_BLINK += 1
                if COUNTER_BLINK >= EYE_AR_CONSEC_FRAMES:
                    cv2.putText(frame, "Blink Detected!", (30, 90),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                if COUNTER_BLINK >= EYE_AR_CONSEC_FRAMES:
                    TOTAL_BLINKS += 1
                COUNTER_BLINK = 0

        # Teeth detection
        roi_gray = gray[y:y+h, x:x+w]
        teeth = teeth_cascade.detectMultiScale(roi_gray, scaleFactor=TEETH_SCALE_FACTOR, minNeighbors=TEETH_MIN_NEIGHBORS)
        for (tx, ty, tw, th) in teeth:
            cv2.rectangle(frame, (x+tx, y+ty), (x+tx+tw, y+ty+th), (0, 255, 255), 2)
            cv2.putText(frame, 'Teeth', (x+tx, y+ty - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)

    # Display frame
    cv2.imshow('Facial Features Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
